# Import library

In [2]:
import pandas as pd
import numpy as np

# Read the data from file

In [3]:
excel_file = "flurry_july_revenue.csv"
flurry = pd.read_csv(excel_file, index_col=False)
#flurry.info()
# show th first 10 rows of the dataset
#flurry.head(10)

# Pre-processing
1. Drop the columns are not required
2. Filter the abnormal figure & correct

In [4]:
# remove any columns from dataframe, where all of the elements of that column are missing.
flurry = flurry.dropna(how='all', axis='columns')
# drop the column 'Id' & 'Date'
flurry = flurry.drop(columns=['Id', 'Date'])

# filter the data
filter1 = flurry['Name']=='Specific_Website_Name'
filter2 = flurry['Ads Requested']<0
flurry[(filter1&filter2)]
flurry[(filter2)]

# correct the negtive figure into positive
flurry['Ads Requested'] = np.where(flurry['Ads Requested']<0, flurry['Ads Requested']*(-1), flurry['Ads Requested'])
#flurry

# Get all website name, not duplicate

In [5]:
website = pd.unique(flurry['Name'])
website = pd.DataFrame(website, columns=['website'])
website = website.sort_values(by=['website'])
website = website.reset_index(drop=True)
#website

# Sum up the figure for specific columns

In [6]:
# convert currency to float in order to sum up
# str[1:]: ignore the first alphabet of the string
flurry['Revenue'] = flurry['Revenue'].str[1:].astype(float)

# sum up "Revenue", "Impression", "Clicks", "Ads Requested", "Ads Received" according to websites' Name
revenue = flurry.groupby(['Name'])['Revenue'].agg('sum')
impression = flurry.groupby(['Name'])['Impressions'].agg('sum')
clicks = flurry.groupby(['Name'])['Clicks'].agg('sum')
ads_request = flurry.groupby(['Name'])['Ads Requested'].agg('sum')
ads_receive = flurry.groupby(['Name'])['Ads Received'].agg('sum')

# combine three columns into dataframe "summary"
summary = pd.concat([revenue, impression, clicks, ads_request, ads_receive], axis=1).reset_index()

# Calculate remain columns

In [7]:
# eCPM(=CPM)=(Revenue*1000)/Impression
ecpm = np.where(impression<=0, 0.0, (revenue*1000)/impression)
ecpm = pd.DataFrame(ecpm)
summary.insert(3, 'eCPM', ecpm)

# fillrate=Ads received/Ads requested
fillrate = np.where(ads_request<=0, 0.0, ads_receive/ads_request)
fillrate = pd.DataFrame(fillrate)
summary.insert(4, 'Fill Rate', fillrate)

# Impression Fill Rate: Impressions/Ads Requested
impression_fillrate = np.where(ads_request<=0, 0.0, impression/ads_request)
impression_fillrate = pd.DataFrame(impression_fillrate)
summary.insert(8, 'Impression Fill Rate', impression_fillrate)

# CTR: Click/Impression
ctr = np.where(impression==0, 0.0, clicks/impression)
ctr = pd.DataFrame(ctr)
summary.insert(9, 'CTR', ctr)

# Format the summary data

In [8]:
# Revenue
summary['Revenue'] = round(summary['Revenue'], 2)
# convert to "Revenue" to string and +"$"
summary['Revenue'] = np.where(summary['Revenue']<0, '-$'+summary['Revenue'].astype(str).str[1:], '$'+summary['Revenue'].astype(str))

# eCPM
summary['eCPM'] = round(summary['eCPM'], 2)
summary['eCPM'] = np.where(summary['eCPM']<0, '-$'+summary['eCPM'].astype(str).str[1:], '$'+summary['eCPM'].astype(str))

# convert "Fill Rate", "Impression Fill Rate", "CTR" into percentage
# {0:.2f}: {[argument_index_or_keyword]:[width][.precision][type]}
summary['Fill Rate'] = pd.Series(["{0:.2f}%".format(val*100) for val in summary['Fill Rate']], index=summary.index)
summary['Impression Fill Rate'] = pd.Series(["{0:.2f}%".format(val*100) for val in summary['Impression Fill Rate']], index=summary.index)
summary['CTR'] = pd.Series(["{0:.2f}%".format(val*100) for val in summary['CTR']], index=summary.index)

#summary

# Output an excel file

In [9]:
# Create a Pandas Excel writer using xlsxwriter as engine
writer = pd.ExcelWriter('flurry_july_revenue_monthly.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel Object
summary.to_excel(writer, index=False, sheet_name='report')

# Get the workbook & worksheet object from xlsxwriter
workbook = writer.book
worksheet = writer.sheets['report']

# Create & Add header format
header_bold = workbook.add_format({'bold':True})

# Set the format of the title row
worksheet.set_row(0, None, header_bold)

# Close the Pandas Excel writer and output the Excel file.
writer.save()